<a href="https://colab.research.google.com/github/jan-kreischer/UZH_ML4NLP/blob/main/Project-05/ex05_sent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 5 - Sequence and Sentiment Classification using Transformers
## Part 2: Resource Limited Competition: Sentiment Analysis
In this part of the project, we are trying out different Transformer models as an alternative to BERT.  
The goal is to fine-tune a Transformer model for the IMDB Binary Sentiment classification task.  

## 1. Setup
### 1.1 Dependencies
Disclaimer: The output of cells which do not produce not helpful output (for example the pip install comands) were cleared to make the notebook easier to read

In [ ]:
!pip install datasets transformers sklearn simpletransformers


### 1.2 Imports

In [3]:
import datasets
from datasets import load_dataset
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures, Trainer, TrainingArguments
from sklearn.preprocessing import LabelEncoder

# Misc
import os
import shutil
import csv
import re
from io import StringIO
import requests
import string
import numpy as np
import matplotlib.pyplot as plt  
import seaborn as sn

# Pandas
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

# Warnings
import warnings
warnings.filterwarnings('ignore')

# Keras
import keras.preprocessing
from keras.models import Sequential
from keras.layers import Conv1D, GlobalMaxPooling1D, AveragePooling1D, Dense, Dropout, Activation, Embedding
from keras import backend as K
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

# tensorflow
import tensorflow as tf

# Torch
import torch

# Sklearn
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score,f1_score

# simpletransformers
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging

### 1.3 Constants

In [4]:
TRAIN_RANGE=(10000,15000)
TEST_RANGE=(11500,13500)
DATASET_URL = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

### 1.4 Environment
We check if the environment we are using is properly setup, such that we are using GPU for training our models.

In [ ]:
# Check if device supports CUDA interface
CUDA = torch.cuda.is_available()
# Make program run on gpu (cuda:0) if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu:0')
torch.cuda.set_device(device)
print('Using device:', device)

Using device: cuda:0


In [ ]:
# Check and print information about available GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Nov 30 01:58:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    30W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Get GPU name
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-09cdc3b9-a810-eb0c-1470-11f3b63223b6)


In [ ]:
# Check Memory
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


## Import


Here we import the data from the Stanford Repository.

In [5]:
# Loading the IMDB sentiment classification dataset
dataset = tf.keras.utils.get_file(fname="aclImdb_v1.tar.gz", 
                                  origin=DATASET_URL,
                                  untar=True,
                                  cache_dir='.',
                                  cache_subdir='')

84140032/84125825 [==============================] - 5s 0us/step


After download the data from the above link, we found that the directory sturcture is like this:

main_directory/                 
...train/                
......a_text_1.txt                
......a_text_2.txt                
...test/                
......a_text_1.txt                
......a_text_2.txt                
...unsup/                
......                



In [10]:
main_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
train_dir = os.path.join(main_dir, 'train')
test_dir = os.path.join(main_dir, 'test')
# We formalize the path to the main directory and its subdirectory. 
# We also remove the "unsup" directory which contains unlabeled reviews for unsupervised learning.
#remove_dir = os.path.join(train_dir, 'unsup')
#shutil.rmtree(remove_dir)

In [7]:
# read data into dataframe: train_data. According to the requeirement, we read train[10000:15000] as train_data.
train = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=25000,
    shuffle=False,
    validation_split=0)

for i in train.take(1):
  train_feat = i[0].numpy()
  train_lab = i[1].numpy()

train = pd.DataFrame([train_feat, train_lab]).T
train.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
train['DATA_COLUMN'] = train['DATA_COLUMN'].str.decode("utf-8")
train_data=train[TRAIN_RANGE[0]:TRAIN_RANGE[1]]

Found 25000 files belonging to 2 classes.


In [11]:
# similarly read the test data into dataframe. According to the requeirement, we read test[11500:13500] as test_data.
test = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/test',
    batch_size=25000,
    shuffle=False,
    validation_split=0)

for i in test.take(1):
  test_feat = i[0].numpy()
  test_lab = i[1].numpy()

test = pd.DataFrame([test_feat, test_lab]).T
test.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
test['DATA_COLUMN'] = test['DATA_COLUMN'].str.decode("utf-8")
test_data=test[TEST_RANGE[0]:TEST_RANGE[1]]

Found 25000 files belonging to 2 classes.


In [12]:
print("Train_data has a shape of {}. \n\n The number of positive(1) and negative(0) reviews are:\n {}".format(
    train_data.shape,train_data['LABEL_COLUMN'].value_counts()))

Train_data has a shape of (5000, 2). 

 The number of positive(1) and negative(0) reviews are:
 1    2500
0    2500
Name: LABEL_COLUMN, dtype: int64


In [13]:
print("Test_data has a shape of {}. \n\n The number of positive(1) and negative(0) reviews are:\n {}".format(
    test_data.shape,test_data['LABEL_COLUMN'].value_counts()))

Test_data has a shape of (2000, 2). 

 The number of positive(1) and negative(0) reviews are:
 1    1000
0    1000
Name: LABEL_COLUMN, dtype: int64


In [14]:
train_data

,DATA_COLUMN,LABEL_COLUMN
10000,"First, the CGI in this movie was horrible. I w...",0
10001,The film is about a sabretooth on the lose at ...,0
10002,Everything about this film is hog wash. Pitifu...,0
10003,Spoilers will be in this. The movie could have...,0
10004,Three giant sabretooth tigers(..created in a l...,0
...,...,...
14995,The minute I started watching this I realised ...,1
14996,i really loved this version of Emma the best. ...,1
14997,Until the 1990s there had never been a film ba...,1
14998,Old Jane's mannered tale seems very popular th...,1


## Models and Classification Arguements
Here we import some Models and compare their prediction results. 

In [15]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

from sklearn.preprocessing import MultiLabelBinarizer

def f1(predictions, outputs):
    mlb = MultiLabelBinarizer()
    return f1_score(
        mlb.fit_transform(test_data['LABEL_COLUMN']),
        mlb.fit_transform(predictions),
        average='weighted'
    )

### Model 1: distilbert-base-uncased-finetuned-sst-2-english
This model is based on the DistilBERT base model, which is the distilled version of the BERT base model and is later fine-tunned by the Stanford Sentiment Treebank(SST). The Stanford Sentiment Treebank consists of sentences from movie reviews and human annotations of their sentiment. The task is to predict the sentiment of a given sentence. We use the two-way (positive/negative) class split, and use only sentence-level labels.

In [16]:
model_args1 = ClassificationArgs(num_train_epochs=10,
                                use_early_stopping=True,
                                output_dir="outputs1/",
                                overwrite_output_dir=True,
                                 weight_decay=0.01)

model1 = ClassificationModel(
    "distilbert", 
    "distilbert-base-uncased-finetuned-sst-2-english", 
    args=model_args1,
    num_labels=2,
    weight=[1,1]
)

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

In [17]:
model1.train_model(train_data)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/5000 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_distilbert_128_2_2


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/625 [00:00<?, ?it/s]

...

Running Epoch 7 of 10:   0%|          | 0/625 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/625 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/625 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of distilbert model complete. Saved to outputs1/.


(6250, 0.10757809510469436)

In [18]:
# Evaluate the model
result, model_outputs, wrong_predictions = model1.eval_model(test_data,acc=accuracy_score,f1_score=f1_score)

print(result)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/2000 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_distilbert_128_2_2


Running Evaluation:   0%|          | 0/250 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.7058478796250932, 'tp': 877, 'tn': 828, 'fp': 172, 'fn': 123, 'auroc': 0.9102195, 'auprc': 0.897877692899084, 'acc': 0.8525, 'f1_score': 0.8560273304050757, 'eval_loss': 1.316734473824501}


{'mcc': 0.7058478796250932, 'tp': 877, 'tn': 828, 'fp': 172, 'fn': 123, 'auroc': 0.9102195, 'auprc': 0.897877692899084, 'acc': 0.8525, 'f1_score': 0.8560273304050757, 'eval_loss': 1.316734473824501}


### Model 2: echarlaix/bert-base-uncased-sst2-acc91.1-d37-hybrid
This model is interesting because it introduce a block pruning methods

In [ ]:
model_args2 = ClassificationArgs(num_train_epochs=10,
                                use_early_stopping=True,
                                output_dir="outputs2/",
                                overwrite_output_dir=True,
                                 weight_decay=0.01)

model2 = ClassificationModel(
    "bert", 
    "echarlaix/bert-base-uncased-sst2-acc91.1-d37-hybrid", 
    args=model_args2,
    num_labels=2,
    weight=[1,1]
)

Downloading:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/352M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/285 [00:00<?, ?B/s]

In [ ]:
model2.train_model(train_data)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/5000 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_bert_128_2_2


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/625 [00:00<?, ?it/s]

...

Running Epoch 7 of 10:   0%|          | 0/625 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/625 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/625 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of bert model complete. Saved to outputs2/.


(6250, 0.11170781915664672)

In [ ]:
# Evaluate the model
result, model_outputs, wrong_predictions = model2.eval_model(test_data,acc=accuracy_score,f1=f1_score)
print(result)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/2000 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_2_2


Running Evaluation:   0%|          | 0/250 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.6804410687651714, 'tp': 858, 'tn': 822, 'fp': 178, 'fn': 142, 'auroc': 0.9164924999999999, 'auprc': 0.908275267636448, 'acc': 0.84, 'f1': 0.8428290766208252, 'eval_loss': 1.5012480400204657}


{'mcc': 0.6804410687651714, 'tp': 858, 'tn': 822, 'fp': 178, 'fn': 142, 'auroc': 0.9164924999999999, 'auprc': 0.908275267636448, 'acc': 0.84, 'f1': 0.8428290766208252, 'eval_loss': 1.5012480400204657}


In [ ]:
predictions, raw_outputs = model2.predict(["This movie is great"])
print(predictions)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[1]


In [ ]:
predictions, raw_outputs = model2.predict(["This movie sucks"])
print(predictions)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[0]


### Model 3: siebert/sentiment-roberta-large-english
This model is a fine-tuned checkpoint of RoBERTa-large ([Liu et al. 2019](https://arxiv.org/pdf/1907.11692.pdf)). It enables reliable binary sentiment analysis for various types of English-language text. According to [Liu et al. 2019](https://arxiv.org/pdf/1907.11692.pdf), this model has improved in the following 4 aspects:
 - (1) training the model longer, with bigger batches,
over more data; 
 - (2) removing the next sentence
prediction objective; 
 - (3) training on longer sequences; and 
 - (4) dynamically changing the masking pattern applied to the training data.    
               
They also collect a large new dataset (CC-NEWS) of comparable size to other privately used datasets, to better control for training set size effects.


In [ ]:
model_args3 = ClassificationArgs(num_train_epochs=10,
                                use_early_stopping=True,
                                output_dir="outputs3/",
                                overwrite_output_dir=True,
                                 weight_decay=0.01)

model3 = ClassificationModel(
    "roberta", 
    "siebert/sentiment-roberta-large-english", 
    args=model_args3,
    num_labels=2,
    weight=[1,1]
)

Downloading:   0%|          | 0.00/687 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256 [00:00<?, ?B/s]

In [ ]:
model3.train_model(train_data)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/5000 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_roberta_128_2_2


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/625 [00:00<?, ?it/s]

Running Epoch 1 of 10:   0%|          | 0/625 [00:00<?, ?it/s]

Running Epoch 2 of 10:   0%|          | 0/625 [00:00<?, ?it/s]

Running Epoch 3 of 10:   0%|          | 0/625 [00:00<?, ?it/s]

In [ ]:
# Evaluate the model
result, model_outputs, wrong_predictions = model3.eval_model(test_data,acc=accuracy_score,f1=f1_score)
print(result)

In [ ]:
predictions, raw_outputs = model3.predict(["This movie is great"])
print(predictions)

In [ ]:
predictions, raw_outputs = model3.predict(["This movie sucks"])
print(predictions)



```
`# This is formatted as code`
```

### Model 4: gchhablani/bert-base-cased-finetuned-sst2
Compared to the bert-base-cased model, this model replaces the self-attention sublayers with simple linear transformations that "mix" input tokens. They show that Transformer encoder can be sped up, with limited accuracy costs. In (this paper[https://arxiv.org/abs/2105.03824]), they showed that these linear mixers, along with standard nonlinearities in feed-forward layers, prove competent at modeling semantic relationships in several text classification tasks. 


In [ ]:
model_args4 = ClassificationArgs(num_train_epochs=10,
                                use_early_stopping=True,
                                output_dir="outputs4/",
                                overwrite_output_dir=True,
                                 weight_decay=0.01)

model4 = ClassificationModel(
    "bert", 
    "gchhablani/bert-base-cased-finetuned-sst2", 
    args=model_args4,
    num_labels=2,
    weight=[1,1]
)

In [ ]:
model4.train_model(train_data)

In [ ]:
# Evaluate the model
result, model_outputs, wrong_predictions = model4.eval_model(test_data,acc=accuracy_score,f1=f1_score)
print(result)

In [ ]:
predictions, raw_outputs = model4.predict(["This movie is great"])
print(predictions)

In [ ]:
predictions, raw_outputs = model4.predict(["This movie sucks"])
print(predictions)